In [46]:
import pymupdf


class DocumentParser:

    def parse(self, file_path: str) -> str:
        if file_path.endswith(".pdf"):
            doc = pymupdf.open(file_path)
            text = ""
            for page in doc:
                text += page.get_text().replace('\u200b', '')
            doc.close()
            return text
        else:
            raise ValueError("File is not a PDF")

In [47]:
syllabus = DocumentParser().parse("./syllabus.pdf")
guidelines = DocumentParser().parse("./persuasive-guidelines.pdf")

In [48]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate


In [49]:
class_context = [
    {"document_name": "Syllabus", "document_content": syllabus},
    {"document_name": "Guidelines for Persuasive Writing", "document_content": guidelines}
    ]

class_document_template = """======= CLASS DOCUMENT =======
DOCUMENT NAME: {document_name}
DOCUMENT CONTENT:\n{document_content}"""

class_context_prefix = "Here are documents from the teacher's class. Use them to extract information about the class and the teacher's goals and teaching style."

class_document_prompt = PromptTemplate.from_template(class_document_template)

class_context_prompt = FewShotPromptTemplate(
    example_prompt=class_document_prompt,
    examples=class_context,
    prefix=class_context_prefix,
    suffix="",
    input_variables=[]
)

# print(class_context_prompt.format())

In [58]:
onboarding = [
    {
        "question": "What is your most high priority goal for your class?",
        "response": "As the teacher of this persuasive writing class, my highest priority goal is to ensure that each student develops the ability to craft clear, compelling, and well-organized arguments. I want my students to not only understand the fundamental components of persuasive writing but also to feel confident in their ability to express their viewpoints effectively. By the end of this course, I aim for every student to be able to construct persuasive essays that are both logical and impactful, supported by strong evidence and presented with a polished writing style. My ultimate goal is to empower students with the skills they need to communicate persuasively in any context, both inside and outside the classroom."
    },
    {
        "question": "How do you describe your overall teaching style, particularly in relation to feedback and student interaction?",
        "response": "My overall teaching style is interactive and student-centered, with a strong emphasis on providing constructive feedback and fostering a supportive learning environment. I believe in the importance of active engagement, so I encourage class discussions, group activities, and peer reviews to help students learn from each other and build their confidence in expressing their ideas. When it comes to feedback, I strive to be thorough and specific, highlighting both strengths and areas for improvement in each student's work. My goal is to guide students in their learning journey by offering actionable suggestions that help them refine their writing skills. I make sure to balance critiques with positive reinforcement, ensuring that students feel motivated and supported as they develop their abilities."
    }
]

onboarding_qa_template = """======= ONBOARDING Q&A =======
QUESTION: {question}
TEACHER RESPONSE: {response}"""

onboarding_qa_prompt = PromptTemplate.from_template(onboarding_qa_template)
onboarding_prefix = "The teacher has been interviewed to learn more about them."

onboarding_prompt = FewShotPromptTemplate(
    example_prompt=onboarding_qa_prompt,
    examples=onboarding,
    prefix=onboarding_prefix,
    suffix="",
    input_variables=[]
)

# print(onboarding_prompt.format())

In [65]:
teacher_model_base_compose_template = """
TASK: {task_instruction}

----- INFORMATION ABOUT THE TEACHER -----

{class_context_prompt}
{onboarding_prompt}

---------------------------

{format_instruction}
"""

teacher_model_base_compose_prompt = PromptTemplate.from_template(teacher_model_base_compose_template)

input_prompts = [
    ("class_context_prompt", class_context_prompt),
    ("onboarding_prompt", onboarding_prompt)
]

teacher_model_base_prompt = PipelinePromptTemplate(
    final_prompt=teacher_model_base_compose_prompt,
    pipeline_prompts=input_prompts,
    input_variables=["task_instruction", "format_instruction"]
)

teacher_model_base_inputs = {
    "task_instruction": "The goal is to generate a persona of this teacher to help with impersonating them. Here is some info on the teacher for you to use.",
    "format_instruction": "Please format the output as a coherent paragraph."
}

teacher_model_base_prompt_str = teacher_model_base_prompt.format(**teacher_model_base_inputs)
# print(teacher_model_base_prompt_str)

# store as text file
with open("teacher_model_base_prompt.txt", "w") as f:
    f.write(teacher_model_base_prompt_str)

## Teacher Model Update Prompt

In [52]:
teacher_model_base = "Ms. [Instructor Name] is a dedicated and interactive teacher who prioritizes helping students develop the ability to craft clear, compelling, and well-organized arguments in persuasive writing. She believes in a student-centered approach, encouraging active engagement through class discussions, group activities, and peer reviews to foster a supportive learning environment. Ms. [Instructor Name] is committed to providing constructive feedback that is thorough and specific, aiming to highlight both strengths and areas for improvement in each student's work. Her goal is to guide students in their learning journey by offering actionable suggestions that help them refine their writing skills while balancing critiques with positive reinforcement to ensure students feel motivated and confident. Ultimately, she strives to empower her students with the skills needed to communicate persuasively in any context, both inside and outside the classroom."


In [56]:
feedback_examples = [
    {
        "highlighted_text": "Homework has long been a topic of debate among students, parents, and educators. Some people argue that homework is essential for reinforcing what students learn in school, while others believe it places unnecessary stress on children. In my opinion, students should not have homework because it limits their time for other important activities, causes stress, and is not always effective in improving learning.",
        "feedback_text": "Your introduction sets the stage for a compelling essay by clearly presenting the topic of homework and the different viewpoints surrounding it. Your thesis statement is strong and clearly outlines your stance against homework, as well as the main arguments you will discuss. To enhance your introduction, consider adding a hook that grabs the reader's attention right from the start. Additionally, you might provide a brief example or statistic to underscore the significance of the homework debate, making your introduction even more engaging."
    },
    {
        "highlighted_text": "Secondly, homework can be a major source of stress for students. The pressure to complete assignments on time and perform well can lead to anxiety and sleep deprivation. For example, a student who struggles with math might spend hours trying to solve problems, only to feel frustrated and exhausted. This kind of stress is unhealthy and can negatively impact a student's overall well-being and academic performance. Reducing or eliminating homework could help alleviate this stress, allowing students to approach their studies with a clearer and more focused mind.",
        "feedback_text": "Your argument effectively highlights the negative impact of homework on students' mental health, using a specific example to illustrate your point. The connection between stress, anxiety, and academic performance is clearly articulated, making your argument compelling. To strengthen this paragraph, consider adding a statistic or expert opinion to further support your claim and enhance the credibility of your argument."
    }
]

feedback_example_template = """======= FEEDBACK EXAMPLE =======
STUDENT WRITING: {highlighted_text}
TEACHER FEEDBACK: {feedback_text}"""

feedback_example_prompt = PromptTemplate.from_template(feedback_example_template)

feedback_examples_prefix = "Here are examples of student writing and feedback provided by this teacher."

few_shot_feedback_prompt = FewShotPromptTemplate(
    example_prompt=feedback_example_prompt,
    examples=feedback_examples,
    prefix=feedback_examples_prefix,
    suffix="",
    input_variables=[]
)

# print(few_shot_feedback_prompt.format())

In [57]:
teacher_model_base_input_template = """
Here is the existing teacher persona for this teacher.
TEACHER PERSONA: {teacher_model_base}
"""

teacher_model_base_input_prompt = PromptTemplate.from_template(teacher_model_base_input_template)

teacher_model_base_input = {
    "teacher_model_base": teacher_model_base
}

teacher_model_base_input_str = teacher_model_base_input_prompt.format(**teacher_model_base_input)

# print(teacher_model_base_input_str)

In [55]:
teacher_model_update_compose_template = """
{task_intro}

----- FEEDBACK EXAMPLES -----
{feedback_examples_prompt}

----- CURRENT TEACHER PERSONA -----
{teacher_model_base_prompt}

---------------------------
{task_outro}

{format_instruction}
"""

teacher_model_update_compose_prompt = PromptTemplate.from_template(teacher_model_update_compose_template)

input_prompts = [
    ("feedback_examples_prompt", few_shot_feedback_prompt),
    ("teacher_model_base_prompt", teacher_model_base_input_prompt)
]

teacher_model_update_prompt = PipelinePromptTemplate(
    final_prompt=teacher_model_update_compose_prompt,
    pipeline_prompts=input_prompts,
    input_variables=["task_intro", "task_outro", "format_instruction"]
)

teacher_model_update_inputs = {
    "task_intro": "The goal is to update the existing teacher persona based on the feedback examples provided.",
    "task_outro": "Please use the feedback examples to refine and enhance the teacher persona. Be sure to describe how this teacher provides feedback to students.",
    "format_instruction": "Please format the output as a coherent paragraph.",
    **teacher_model_base_input
}

teacher_model_update_prompt_str = teacher_model_update_prompt.format(**teacher_model_update_inputs)

# store as text file
with open("teacher_model_update_prompt.txt", "w") as f:
    f.write(teacher_model_update_prompt_str)